In [2]:
import batch_data_support_general as bds_gen
import importlib
import inspect
import itertools
import logging
import matplotlib.pyplot as plt
import model_attributes as ma
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_electricity as ml
import model_energy as me
import model_socioeconomic as se
import setup_analysis as sa
import support_classes as sc
import support_functions as sf
import numpy as np
import os, os.path
import pandas as pd
import pycountry
import re
import time
from typing import *
import warnings

warnings.filterwarnings("ignore")


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2287: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


In [3]:
regions = sc.Regions(sa.model_attributes)
time_periods = sc.TimePeriods(sa.model_attributes)

In [5]:
dir_repo_data = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data"
t0 = time.time()
df_complete, dict_modvar_to_fields, dict_modvar_to_ordered_cats = read_inputs_from_repository(dir_repo_data, None, sa.model_attributes)
sf.get_time_elapsed(t0)

appended frac_agrc_cereals_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_fibers_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_fruits_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_herbs_and_other_perennial_crops_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_nuts_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_other_annual_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_other_woody_perennial_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_pulses_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_rice_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_sugar_cane_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_tubers_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_vegetables_and_vines_cl2_dry
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_bevs_and_spices_cl1_temperate
(8815, 2)
(8815, 3)
(8815, 3)


appended frac_agrc_cereals_cl1_temperate
(8815, 2)
(8815, 3)
(8815

348.05

In [32]:
df_complete.drop(["index"], axis = 1, inplace = True) if ("index" in df_complete.columns) else None
df_complete["year"] = df_complete["year"].astype(int)
df_complete = df_complete[~df_complete["iso_code3"].isna()]
df_complete = df_complete[~df_complete["frac_agrc_bevs_and_spices_cl2_dry"].isna()].reset_index(drop = True)

In [55]:
df_complete[df_complete["iso_code3"].isin(["BRA"])][fields_index + ["gdp_mmm_usd"]]

,iso_code3,year,gdp_mmm_usd
1066,BRA,2010,1972.551979
1067,BRA,2011,2064.430164
1068,BRA,2012,2156.308348
1069,BRA,2013,2248.186533
1070,BRA,2014,2340.064717
1071,BRA,2015,2431.942902
1072,BRA,2016,2421.772225
1073,BRA,2017,2411.601547
1074,BRA,2018,2401.430870
1075,BRA,2019,2391.260192


In [42]:

#df_complete.dropna()
df2020 = df_complete[df_complete["year"].isin([2020])]
df20202 = df2020[["iso_code3", "year"] + [x for x in df2020.columns if "scoe" in x]]

check = df20202[df20202["iso_code3"].isin(["CHL"])]

for x in check.columns:
    if len(check[x].dropna()) == 0:
        print(x)

In [452]:
fields_index = ["iso_code3", "year"]
#df_var[1].set_index(fields_index, inplace = True)

In [456]:
df_var_cur = df_var[1]

In [442]:
df_var = pd.concat(df_var, axis = 0) if (len(df_var) > 0) else None

In [458]:
#df_var[["iso_code3", "year"]]

In [459]:
inds_prev = df_var[0].index

In [481]:
fp_read2 = "/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data/IPPU/net_imports_cement_clinker_tonne/input_to_sisepuede/historical/net_imports_cement_clinker_tonne.csv"

fp_read == fp_read2

False

In [486]:
df = pd.read_csv(fp_read)
dict_rnm = dict((x, x.lower()) for x in df.columns)
df.rename(columns = dict_rnm, inplace = True)
df[fields_index].drop_duplicates().shape# == df[fields_index].shape


(5431, 2)

In [387]:
df = pd.read_csv(fp_read)
df[
    df["iso_code3"].isin(["EGY"]) 
    & df["Year"].isin([2019])
]

,Year,Nation,iso_code3,population_gnrl_urban
1768,2019,"Egypt, Arab Rep.",EGY,3.996680e+07


In [301]:
fp_read_2 = '/Users/jsyme/Documents/Projects/git_jbus/sisepuede_data/SocioEconomic/population_gnrl_urban/input_to_sisepuede/historical/population_gnrl_urban.csv'
df_2 = pd.read_csv(fp_read_2)
df_2[
    df_2["iso_code3"].isin(["EGY"]) 
    & df_2["Year"].isin([2019])
]



,Year,Nation,iso_code3,population_gnrl_urban
1505,2019,"Egypt, Arab Rep.",EGY,4.289582e+07


KeyboardInterrupt: 

In [ ]:
df2020 = df[df["year"] == 2020]
vals = []
for k in df.columns:
    tt = len(df[k].dropna())
    
    if tt < len(df):
        vals.append(k)
df[df[vals[0]].isna()]["iso_code3"].unique()

In [90]:
df[
    df["yf_agrc_rice_tonne_ha"].isna()
]

,year,iso_code3,frac_agrc_bevs_and_spices_cl2_dry,frac_agrc_cereals_cl2_dry,frac_agrc_fibers_cl2_dry,frac_agrc_fruits_cl2_dry,frac_agrc_herbs_and_other_perennial_crops_cl2_dry,frac_agrc_nuts_cl2_dry,frac_agrc_other_annual_cl2_dry,frac_agrc_other_woody_perennial_cl2_dry,...,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,2010,ABW,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,AFG,0.994548,0.994548,0.994548,0.994548,0.994548,0.994548,0.994548,0.994548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,AGO,0.615825,0.615825,0.615825,0.615825,0.615825,0.615825,0.615825,0.615825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,ALB,0.320181,0.320181,0.320181,0.320181,0.320181,0.320181,0.320181,0.320181,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,AND,0.117647,0.117647,0.117647,0.117647,0.117647,0.117647,0.117647,0.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9058,2050,SXM,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9061,2050,TCA,0.086397,0.086397,0.086397,0.086397,0.086397,0.086397,0.086397,0.086397,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9072,2050,TUR,0.887015,0.887015,0.887015,0.887015,0.887015,0.887015,0.887015,0.887015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9084,2050,VGB,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
df_clim = pd.read_csv("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/ref/batch_data_generation/koppen_climate_classifications/climate_fields_by_country.csv")




In [94]:
df_clim[df_clim["iso_alpha_3"].isin(["FRA"])]

,iso_alpha_3,time_period,year,frac_agrc_bevs_and_spices_cl1_temperate,frac_agrc_bevs_and_spices_cl1_tropical,frac_agrc_bevs_and_spices_cl2_dry,frac_agrc_bevs_and_spices_cl2_wet,frac_agrc_cereals_cl1_temperate,frac_agrc_cereals_cl1_tropical,frac_agrc_cereals_cl2_dry,...,frac_frst_secondary_cl1_temperate_nutrient_rich,frac_frst_secondary_cl1_tropical,frac_lndu_grasslands_cl1_temperate,frac_lndu_grasslands_cl1_tropical,frac_lndu_grasslands_cl2_dry,frac_lndu_grasslands_cl2_wet,frac_lndu_other_cl2_dry,frac_lndu_other_cl2_wet,frac_lndu_settlements_cl2_dry,frac_lndu_settlements_cl2_wet


In [95]:
df_clim["iso_alpha_3"].unique()

array(['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM',
       'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD',
       'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA',
       'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN',
       'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB',
       'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DJI', 'DMA', 'DNK', 'DOM',
       'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI',
       'FRO', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIB', 'GIN', 'GMB',
       'GNB', 'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HKG',
       'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IMN', 'IND', 'IRL', 'IRN',
       'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN',
       'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBR',
       'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAC',
       'MAF', 'MAR', 'MCO', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL', 'MKD',
       'MLI', 'MLT',